In [1]:
import numpy as np
import six
import sys
import os
import traceback
import re
import pickle
from copy import deepcopy

from chainer import cuda
from context2vec.common.context_models import Toks
from context2vec.common.model_reader import ModelReader
import sklearn
import pandas as pd
import logging
from scipy.stats import spearmanr
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
import math




[nltk_data] Downloading package stopwords to /home/ql261/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def produce_top_n_simwords(w_filter,context_embed,n_result,index2word):
        #assume that w_filter is already normalized
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
        similarity_scores=[]
        print('producing top {0} simwords'.format(n_result))
        similarity = (w_filter.dot(context_embed)+1.0)/2
        top_words_i=[]
        top_words=[]
        count = 0
        for i in (-similarity).argsort():
                    if xp.isnan(similarity[i]):
                        continue
                    print('{0}: {1}'.format(str(index2word[i]), str(similarity[i])))
                    count += 1
                    top_words_i.append(i)
                    top_words.append(index2word[i])
                    similarity_scores.append(similarity[i])
                    if count == n_result:
                        break

        top_vec=w_filter[top_words_i,:]
        
        return top_vec,np.array(similarity_scores),top_words
    
def top_mutual_sim(top_vec,similarity_scores):

    #normalize the top_vec
    s = np.sqrt((top_vec * top_vec).sum(1))
    s[s==0.] = 1.
    top_vec /= s.reshape((s.shape[0], 1))
    
    # substitutes' similarity to sentence (similarity_scores) as weight matrix to mutual similarity
    max_score=similarity_scores[0]
    similarity_scores=np.array(similarity_scores)
    sim_weights=(similarity_scores+similarity_scores.reshape(len(similarity_scores),1))/2.0
    #weighted by the maximum score in the substitutes (highre max score means the context is more certain about the substitutes)
    sim_weights=(sim_weights/float(sum(sum(sim_weights))))*max_score
    # dot product weighted by substitute probability (sim_weights)
    inf_score=sum(sum(top_vec.dot(top_vec.T)*sim_weights))
    return inf_score

In [43]:
def load_w2salience(w2salience_f,weight_type):
    w2salience={}
    with open(w2salience_f) as f:
        for line in f:
            if line.strip()=='':
                continue
            w,w_count,s_count=line.strip().split('\t')
            if weight_type==INVERSE_W_FREQ:
                w2salience[w]=1/float(w_count)
            elif weight_type==INVERSE_S_FREQ:
                w2salience[w]=math.log(1+84755431/float(s_count))
    return w2salience

def skipgram_context(model,words,pos,weight=None,w2entropy=None):
    context_wvs=[]
    weights=[]
    for i,word in enumerate(words):
        if i != pos: #surroudn context words
            try:
                if weight ==LDA:
                    if word in w2entropy and word in model:
                        print (word,w2entropy[word])
                        weights.append(1/(w2entropy[word]+1.0))
                        context_wvs.append(model[word])
                elif weight in [INVERSE_W_FREQ,INVERSE_S_FREQ]:
                    if word in w2entropy and word in model:
                        print (word,w2entropy[word])
                        weights.append(w2entropy[word])
                        context_wvs.append(model[word])
                else:
                    #equal weights per word
                    context_wvs.append(model[word])
                    weights.append(1.0)
            except KeyError as e:
                print ('==warning==: key error in context {0}'.format(e))
    context_embed=sum(np.array(context_wvs)*np.array(weights).reshape(len(weights),1))#/sum(weights)
    return sum(weights),context_embed #  will be normalized later

def lg_model_out_w2v(top_words,w_target,word2index_target):
        # lg model substitutes in skipgram embedding
        top_vec=[]
        index_list=[]
        for i,word in enumerate(top_words):
            try :
                top_vec.append(w_target[word2index_target[word]])
                index_list.append(i)
            except KeyError as e:
                print (e)
        return np.array(top_vec),index_list
    
def context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    #produce context representation and infromative score for each context
    test_s=test_s.replace(test_w, ' '+test_w+' ')
    print(test_s)
    words=test_s.split()
    pos=words.index(test_w)
    
    score=1.0 #default score
    
    # Decide on the model
    if model_type=='context2vec':
        context_embed= model.context2vec(words, pos)
        context_embed_out=context_embed
    
    elif model_type=='skipgram':
        score,context_embed=skipgram_context(model,words,pos,weight,w2entropy)
        context_embed_out=context_embed
        
    elif model_type=='context2vec-skipgram':
        # context2vec substitutes in skipgram space
        context_embed= model.context2vec(words, pos)
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        top_vec,index_list=lg_model_out_w2v(top_words,w_target,word2index_target) 
        sim_scores=sim_scores[index_list] #weighted by substitute probability
        context_embed_out=sum(top_vec*((sim_scores/sum(sim_scores)).reshape(len(sim_scores),1)))
    else:
        print ('model type {0} not recognized'.format(model_type))
        sys.exit(1)
        
        
    #decide on weight per sentence
    if weight==TOP_MUTUAL_SIM:
        print (weight)
        if word2index_target!=None:
            #context2vec word embedding space neighbours
            top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        #skipgram word embedding space neighbours when context2vec-skipgram
        score=top_mutual_sim(top_vec,sim_scores)
        print (score)

    elif weight=='learned':
        print ('learned not implemented')
    elif weight=='gaussian':
        print ('gaussian not implemented')
    elif weight ==False or weight in [LDA,INVERSE_S_FREQ,INVERSE_W_FREQ]:
        score=score
    else:
        print ('weight mode {0} not recognized'.format(weight))
    return score,context_embed_out

def additive_model(f_w,test_ss,test_w, model_type,model,n_result,w_filter,index2word,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    #produce context representation across contexts using weighted average
    
    context_out=[]
    context_weights=[]
    for test_s in test_ss.split('@@'):
        test_s=test_s.strip()
        #produce context representation with scores
        score,context_embed=context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
        print ('weight is {0}'.format(score))
        context_out.append(context_embed)
        context_weights.append(score)
    
    
    #sum representation across contexts
    context_out=np.array(context_out)
    norm_weights=np.array(context_weights).reshape(len(context_weights),1)/float(sum(context_weights))
    f_w.write(','.join([str(i[0]) for i in norm_weights])+'\n')
    print ('normalized weight: \n  {0}'.format(norm_weights))
    
    if model_type=='skipgram':
        # context representation by weighted sum of all context words in all contexts
        context_avg=sum(context_out)/sum(context_weights)
    else:
        # context represenatation by weighted sum of contexts
        context_avg=sum(norm_weights*context_out)
    
    
    # check new embedding neighbours

    print('producing top {0} words for new embedding'.format(n_result))
    if index2word_target==None:
        top_vec,scores,top_words=produce_top_n_simwords(w,context_avg,n_result,index2word)
    else:
        #print the target space neighbours for context2vec-skipgram
        print (w_target.shape)
        top_vec,scores,top_words=produce_top_n_simwords(w_target,context_avg,n_result,index2word_target)
    
    return context_avg




In [6]:
def filter_w(w,word2index,index2word):
    #filter out words with no letters in, and stopwords
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    index2word_filter={}
    word2index_filter={}
    index_filter2index=[]
    counter=0
    for word in word2index:
            if word not in stopw:
                    index_filter2index.append(word2index[word])
                    word2index_filter[word]=counter
                    index2word_filter[counter]=word
                    counter+=1
    w_filter= w[index_filter2index,:]
    return w_filter,word2index_filter,index2word_filter

def rm_stopw_context(model):
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    
    model={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
    return model




In [7]:
def eval_chimera(chimeras_data_f,context_model,model_type,n_result,w,index2word,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    chimeras_data_dir='/'.join(chimeras_data_f.split('/')[:-1])
    num_sent=chimeras_data_f.split('/')[-1].split('.')[1][1]
    print (chimeras_data_dir)
    print (num_sent)
    with open(chimeras_data_dir+'/weights_{0}_{1}_{2}'.format(num_sent,model_type,str(weight)),'w') as f_w:
        spearmans=[]
        data=pd.read_csv(os.path.join(chimeras_data_f),delimiter='\t',header=None)

        for index, row in data.iterrows():
            golds=[]
            model_predict=[]
            probes=[]
            #compute context representation
            if weight!='learned':
                context_avg=additive_model(f_w,row[1].lower(),'___', model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
            context_avg = context_avg / xp.sqrt((context_avg * context_avg).sum())

            #cosine similarity with probe embedding
            for gold,probe in zip(row[3].split(','),row[2].split(',')):
                try:
                    if index2word_target==None:
                        probe_w_vec=xp.array(w[word2index[probe]])
                    else:
                        probe_w_vec=xp.array(w_target[word2index_target[probe]])
                    probe_w_vec=probe_w_vec/xp.sqrt((probe_w_vec*probe_w_vec).sum())
                    cos=probe_w_vec.dot(context_avg)
                    if xp.isnan(cos):
                        continue
                    else:
                        model_predict.append(cos)
                        golds.append(gold)
                        probes.append(probe)
                except KeyError as e:
                    print ("====warning key error for probe=====: {0}".format(e))
            print ('probes',probes)
            print ('gold',golds)
            print ('model_predict',model_predict)
            sp=spearmanr(golds,model_predict)[0]
            print ('spearman correlation is {0}'.format(sp))
            if not math.isnan(sp):
                spearmans.append(sp)
        print ("AVERAGE RHO:",float(sum(spearmans))/float(len(spearmans)))

In [8]:
# a=np.array([1,2,3])
# b=a.reshape(len(a),1)
# c=(a+b)/2.0
# c=c/(sum(sum(c)))
# # print(sum(sum(c)))
# d=np.array([[1,1,1],[2,2,2],[3,3,3]])
# e=d.dot(d.T)
# print ('e',e)
# print ('c',c)
# e*c
# a
# context_embed= model.context2vec(['cats',',','dogs',',','snakes','are','animals'],2 )
# context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
# produce_top_n_simwords(context_embed=context_embed,index2word=index2word,w_filter=w,n_result=20)

In [47]:
TOP_MUTUAL_SIM='top_mutual_sim'
LDA='lda'
INVERSE_S_FREQ='inverse_s_freq'
INVERSE_W_FREQ='inverse_w_q'
WEIGHT_DICT={0:False,1:TOP_MUTUAL_SIM,2:LDA,3:INVERSE_S_FREQ,4:INVERSE_W_FREQ}


if __name__=="__main__":
    
    #params read in
    if sys.argv[0]=='/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py':
        
        data='./eval_data/data-chimeras/dataset.l4.fixed.test.txt.punct'

        weight=WEIGHT_DICT[1]
        
#         ##context2vec
##         model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params'
#         model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.14'
        
#         model_type='context2vec'
#         context_rm_stopw=0

# ####skipgram
#         model_param_file='../models/wiki_all.model/wiki_all.sent.split.model'
#         model_type='skipgram'
#         context_rm_stopw=1
#         weight='inverse_w_freq'
#         w2salience_f='../corpora/corpora/wiki.all.utf8.sent.split.tokenized.vocab'
#         w2salience_f='../models/lda/w2entropy'


####context2vec-skipgram
#         model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.14,../models/wiki_all.model/wiki_all.sent.split.model'
        model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params,../models/wiki_all.model/wiki_all.sent.split.model'
        model_type='context2vec-skipgram'
        context_rm_stopw=0
    
    else:
        if len(sys.argv) < 6:
            print >> sys.stderr, "Usage: %s <model_param_file> <model_type> <weight:0:False,1:'top_mutual_sim',2:'lda',3:'inverse_s_freq',4:'inverse_w_freq'> <context_rm_stop> <eval_data> <w2salience>"  % (sys.argv[0])
            sys.exit(1)
        
        model_param_file = sys.argv[1]
        model_type=sys.argv[2]
        
        weight=WEIGHT_DICT[int(sys.argv[3])]
        context_rm_stopw=int(sys.argv[4])
        data =sys.argv[5]
        if len(sys.argv)>6:
            w2salience_f=argv[6]
        else:
            w2salience_f=None
    
    
    #gpu setup 
    gpu = -1 # todo: make this work with gpu

    if gpu >= 0:
        cuda.check_cuda_available()
        cuda.get_device(gpu).use()    
    xp = cuda.cupy if gpu >= 0 else np
    
    # logging
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    
    #choose model type
    print ('read model....')
    if model_type=='context2vec':
        #read in model
        
        model_reader = ModelReader(model_param_file)
        w = model_reader.w
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
        w_target=None
        word2index_target=None
        index2word_target=None
    elif model_type=='skipgram':
        model = gensim.models.Word2Vec.load(model_param_file)
        w=deepcopy(model.wv.vectors)
        #vector normalize for probe w embedding
        s = np.sqrt((w * w).sum(1))
        s[s==0.] = 1.
        w /= s.reshape((s.shape[0], 1))
        
        index2word=model.wv.index2word
        word2index={key: model.wv.vocab[key].index for key in model.wv.vocab}
        w_target=None
        word2index_target=None
        index2word_target=None
        
        
    elif model_type=='context2vec-skipgram':
        model_param_context,model_param_w2v=model_param_file.split(',')
        model_reader = ModelReader(model_param_context)
        w = model_reader.w
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
        
        model_w2v = gensim.models.Word2Vec.load(model_param_w2v)
        w_target=model_w2v.wv.vectors
        index2word_target=model_w2v.wv.index2word
        word2index_target={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
    
    
    w2salience=None
    
    #remove stop words in target word space
    print ('filter words for target....')
    w,word2index,index2word=filter_w(w,word2index,index2word)
    if  index2word_target!=None:
        w_target,word2index_target,index2word_target=filter_w(w_target,word2index_target,index2word_target)
    
    #weight

    if weight==TOP_MUTUAL_SIM:
        n_result = 20
    elif weight==LDA:
        print ('load vectors and entropy')
        w2salience=pickle.load(open(w2salience_f))
    elif weight==INVERSE_W_FREQ:
        print ('load w2freq')
        w2salience=load_w2salience(w2salience_f,weight)
    elif weight==INVERSE_S_FREQ:
        print ('load w2freq')
        w2salience=load_w2salience(w2salience_f,weight)


    # remove context stop words
    if int(context_rm_stopw)==1:
        print ('filter words for context....')

        model=rm_stopw_context(model)
        
    

read model....
Reading config file: ../models/context2vec/model_dir/context2vec.ukwac.model.params
Config:  {'config_path': '../models/context2vec/model_dir/', 'model_file': 'context2vec.ukwac.model', 'deep': 'yes', 'drop_ratio': '0.0', 'words_file': 'context2vec.ukwac.words.targets', 'unit': '300'}
filter words for target....


In [51]:
#read in data
# data='./eval_data/data-chimeras/dataset.l6.fixed.test.txt.punct'
if data.split('/')[-2]== 'data-chimeras':
#         weight=None
        eval_chimera(data,model,model_type,20,w,index2word,weight,w2salience,w_target,word2index_target,index2word_target)
        

./eval_data/data-chimeras
6
canned sardines and  ___  between two slices of wholemeal bread and thinly spread flora original .
producing top 20 simwords
sardines: 0.5550946
scallops: 0.55297107
fillets: 0.5502432
halloumi: 0.5501815
eggs: 0.5495906
walnuts: 0.5471799
chickpeas: 0.546566
manioc: 0.54639363
margarine: 0.54610455
prawns: 0.5459013
anchovies: 0.5441009
salmon: 0.5434682
sandwiched: 0.5434074
sauteed: 0.5432048
oxtail: 0.54201144
oysters: 0.5415581
tempeh: 0.5402591
wedges: 0.53997344
cannelloni: 0.5395091
split: 0.5370426
'halloumi'
'cannelloni'
weight mode None not recognized
weight is 1.0
erm ,  ___  , low fat dairy products , incidents of heart disease for those who have an olive oil rich diet .
producing top 20 simwords
dairy: 0.56315917
low-fat: 0.5603089
low-carb: 0.55932075
wheatgerm: 0.5581963
erm: 0.55741954
fat: 0.5571637
soya: 0.55571413
weetabix: 0.5552161
fat-free: 0.5549448
low-salt: 0.5536629
muesli: 0.55134565
complan: 0.550797
cereals: 0.5507624
low-carboh

marinated: 2.7851042071045145
chickpeas: 2.7476275670444728
shallots: 2.742556944785969
salads: 2.7250056177368642
cilantro: 2.7056169277215956
coriander: 2.7038247998030744
stewed: 2.6918453319643834
soups: 2.6709774679948017
lentils: 2.641571244115829
mozzarella: 2.640958853024406
sauces: 2.6385559967765397
braised: 2.621306875650568
almonds: 2.5940574135568077
toppings: 2.584993716476316
manioc: 2.5845950400813313
pickled: 2.5787321404536026
minced: 2.573905825426581
onions: 2.5608099636123454
chilies: 2.5527396253900783
condiments: 2.5361338959052695
('probes', ['rhubarb', 'onion', 'pear', 'strawberry', 'limousine', 'cushion'])
('gold', ['3', '3.86', '4', '3.71', '2.14', '2.29'])
('model_predict', [0.566743691689867, 0.6460365707570784, 0.5330873850304276, 0.5267614774866256, 0.1904421093567073, 0.2524320456124808])
spearman correlation is 0.714285714286
here are a few suggestions: � how many of us know how much water our washing machines  ___  use ?
producing top 20 simwords
actua

phyllaries: 2.117731449270209
rinsed: 2.1051531264042582
marinated: 2.0970257436170363
sepals: 2.0949395928198267
evaporator: 2.091463446628423
sealant: 2.0882787857271206
workpiece: 2.0870751777937357
publicationdate: 2.083142703707363
caramelized: 2.0827309316006284
strainer: 2.0738593327682247
borosilicate: 2.0688682313660633
neoprene: 2.0646689832893155
smokebox: 2.0632199750718248
pelecaniformesfamily: 2.0614660350982454
mandrel: 2.060558909911296
moistened: 2.0559883281722358
griddle: 2.0498138453993127
dryers: 2.047222483974095
hydroxide: 2.044313589604925
breaded: 2.042260236198439
('probes', ['stove', 'microwave', 'kettle', 'cage', 'wastebin', 'leopard'])
('gold', ['2.86', '2.57', '2.14', '1.57', '1.57', '1.71'])
('model_predict', [0.6011002724770698, 0.4112976080435775, 0.43819210356652755, 0.3812229249168539, 0.29941823028697134, 0.24815313023198543])
spearman correlation is 0.753702346348
what never needs sharpening , is completely  ___  safe and is covered by a 25 year gua

sectarian: 0.5497838
loyalist: 0.5473407
anti-israeli: 0.54733646
paramilitary: 0.5446698
unprovoked: 0.5414058
high-profile: 0.54098564
anti-terror: 0.54068524
anti-government: 0.54049146
retaliatory: 0.539699
anti-us: 0.5392031
i.r.a.: 0.53781694
anti-jewish: 0.53765225
islamophobic: 0.5365023
separatist: 0.53617585
treasonable: 0.5359483
armed: 0.53586334
anti-semitic: 0.5356879
well-publicised: 0.53544855
racist: 0.53402865
anti-british: 0.5340177
'anti-israeli'
'high-profile'
'anti-terror'
'anti-government'
'anti-us'
'i.r.a.'
'anti-jewish'
'anti-semitic'
'well-publicised'
'anti-british'
weight mode None not recognized
weight is 1.0
four seconds later , the four rof gosling solid - propellant boost rockets have accelerated the  ___  to mach 2.5 and fallen away .
producing top 20 simwords
rocket: 0.5449529
acceleration: 0.5436807
descent: 0.5432601
stuka: 0.5412865
re-entry: 0.54013145
rockets: 0.5395875
t-33: 0.5381145
afterburner: 0.5378325
change-over: 0.5376468
payload: 0.537530

producing top 20 simwords
dragonforce: 0.5331183
contestants: 0.5321154
everyone: 0.5275527
lordi: 0.52739966
crowds: 0.5263249
snowboarders: 0.52574134
party-goers: 0.5255851
everybody: 0.52524835
spectators: 0.52512056
players: 0.5245283
fans: 0.523403
duels: 0.5227126
non-believers: 0.52241546
revellers: 0.5217359
others: 0.5215377
skaters: 0.52150106
grown-ups: 0.5212984
nobody: 0.5200968
hyenas: 0.5192344
...they: 0.5187852
'party-goers'
'non-believers'
'grown-ups'
'...they'
weight mode None not recognized
weight is 1.0
these remarks apply only to cases in which horns and  ___  are used alone , or almost alone .
producing top 20 simwords
horns: 0.5501156
limbs: 0.54817325
appendages: 0.54719317
viols: 0.5442467
instruments: 0.54376715
organs: 0.5437599
torsos: 0.54144317
bells: 0.5412248
vertebrae: 0.54077446
microphones: 0.5397344
bones: 0.5382477
bridles: 0.53761387
clavichords: 0.53707063
ornaments: 0.5353724
snares: 0.5338612
basses: 0.5332301
arrows: 0.53320795
strings: 0.533

...

producing top 20 simwords
lumberjack: 0.554999
kid: 0.54848385
tambourine: 0.54427844
clown: 0.54246324
guy: 0.54168886
heckler: 0.5415482
guitar: 0.5412581
phonograph: 0.5403575
wombat: 0.539603
theremin: 0.53931004
dancer: 0.5381444
baddy: 0.5377803
fiddle: 0.5375904
seagull: 0.53746593
double-bass: 0.53732497
dud: 0.5372553
drummer: 0.5369642
ukulele: 0.53678536
bazooka: 0.53628504
giraffe: 0.53627753
'baddy'
'double-bass'
weight mode None not recognized
weight is 1.0
p this is a document covering the history of the  ___  from medieval to present times .
producing top 20 simwords
commandery: 0.56228024
dobunni: 0.55971277
church: 0.5560308
celts: 0.55078787
pottery: 0.5490281
scriptorium: 0.54854184
crusades: 0.5476549
normans: 0.54573363
burgh: 0.54475963
monasteries: 0.5446415
vicus: 0.5443245
c17th: 0.54423296
picts: 0.5441418
papacy: 0.54352057
diocese: 0.5431294
psalter: 0.54289603
priory: 0.54277295
romans: 0.54208475
belgae: 0.5418553
ismailis: 0.5415712
'c17th'
weight mode N

seagull: 0.53845006
boulder: 0.538145
crane: 0.5373701
sombrero: 0.5363744
biped: 0.53527904
chandelier: 0.53461057
colander: 0.53426284
gasometer: 0.533705
spanner: 0.533435
turnip: 0.53262097
keg: 0.5325419
firework: 0.5320871
statue: 0.53141654
stalactite: 0.53133386
weight mode None not recognized
weight is 1.0
if th ' guts ' d been griping and grinding like as if a  ___  were twisting there , thee ' ud have done the same .
producing top 20 simwords
hammer: 0.5486784
dey: 0.54199904
bugler: 0.54057413
cheater: 0.5390655
wolf: 0.5385177
man: 0.5372212
nigger: 0.5365937
knife: 0.5364834
cock: 0.5361179
cow: 0.5360174
faggot: 0.5358232
bastard: 0.5356069
corpse: 0.53481704
cudgel: 0.5341959
grindstone: 0.5338447
chinaman: 0.5337992
madman: 0.533499
needle: 0.53349376
drumstick: 0.53284025
lemming: 0.53249955
weight mode None not recognized
weight is 1.0
use a  ___  or mandolin to shred apple and parsnip , mix with lemon juice and season well .
producing top 20 simwords
tambourine: 0.5

producing top 20 simwords
products: 0.5486432
designs: 0.5434315
fragrances: 0.5412872
qualities: 0.5379932
colours: 0.53314674
fabrics: 0.53220755
effects: 0.5316606
ingredients: 0.52965397
formulations: 0.5296443
traits: 0.52953416
features: 0.52940565
benefits: 0.5288206
flavours: 0.5288168
profiles: 0.52837145
ranges: 0.5279562
garments: 0.5276394
lotions: 0.5263926
supplements: 0.52593064
shampoos: 0.5254932
colors: 0.5254374
weight mode None not recognized
weight is 1.0
almond biscuits will be moister if you grind nuts freshly -- grate blanched almonds in a cylindrical - shaped  ___  .
producing top 20 simwords
saucepan: 0.54844004
flan: 0.54833496
casserole: 0.5477922
ramekin: 0.546086
dish: 0.5451895
meringue: 0.5441833
bowl: 0.5438301
sponge: 0.54193246
pot: 0.54143035
rind: 0.539658
jar: 0.5389445
jug: 0.5381263
capsule: 0.53760415
shape: 0.5372987
tortilla: 0.5369896
platter: 0.53609324
rectangle: 0.5348682
tin: 0.53457755
skillet: 0.5338888
spoon: 0.53262985
'ramekin'
weigh

producing top 20 simwords
furiously: 0.5566553
helplessly: 0.54811794
noisily: 0.5453265
mercilessly: 0.544319
ferociously: 0.5418023
ineffectually: 0.5403693
unmercifully: 0.53902173
uncontrollably: 0.53892547
moodily: 0.5387999
jauntily: 0.53820145
aimlessly: 0.53809106
incessantly: 0.53787816
drunkenly: 0.53768593
erratically: 0.5368717
maniacally: 0.53677714
menacingly: 0.5349512
nervously: 0.5346048
frantically: 0.53443164
inanely: 0.5343296
silently: 0.5342665
'moodily'
'jauntily'
'inanely'
weight mode None not recognized
weight is 1.0
cajuns love to eat these  ___  with baked ham which are popular at thanksgiving .
producing top 20 simwords
pies: 0.55362093
puddings: 0.55126476
eggs: 0.54848343
sandwiches: 0.5472988
dishes: 0.5460992
bagels: 0.5453611
pancakes: 0.5450438
oatcakes: 0.5437394
buns: 0.5436072
trifles: 0.54304254
confections: 0.54202527
vegetables: 0.54188454
dinners: 0.5418123
salads: 0.54177535
cheeses: 0.5417442
delicacies: 0.5407388
tarts: 0.5403873
cakes: 0.540

trees: 0.5528835
bushes: 0.5524758
treetops: 0.54290503
underbrush: 0.5421692
tree-tops: 0.5407574
banisters: 0.5384155
railings: 0.5365775
hedge: 0.5363194
gates: 0.5358408
raindrops: 0.5357801
fences: 0.53440285
chinks: 0.53430593
hedgerow: 0.5332581
boughs: 0.53272736
nettles: 0.532424
fields: 0.53224975
thicket: 0.53194845
poplars: 0.5308379
portaloos: 0.53083587
reeds: 0.53007126
'tree-tops'
'portaloos'
weight mode None not recognized
weight is 1.0
he experimented with growing sweet  ___  , tobacco and grapes and made wine out of everything .
producing top 20 simwords
fruit: 0.5542632
chestnuts: 0.55171114
oranges: 0.55026674
fruits: 0.54653645
lemons: 0.5462147
strawberries: 0.5428877
apples: 0.5415504
grapes: 0.5414972
corn: 0.5407566
herbs: 0.54001063
syrups: 0.5388872
liquors: 0.5385553
oils: 0.5379318
nuts: 0.5375703
gourds: 0.53726393
bananas: 0.53708756
gins: 0.5369027
raisins: 0.5368748
cherries: 0.5364488
perfumes: 0.53615546
weight mode None not recognized
weight is 1.0
